# 📊 SmartCampus Parking Predictor (DNN)

**Project Goal:** Developing a Deep Neural Network (DNN) model to accurately predict real-time parking occupancy rates at a university campus based on time-series, academic, and environmental variables.

**Key Achievements:**
* **Model Accuracy:** Achieved 98.81% accuracy (MAE-based).
* **Core Technology:** TensorFlow/Keras DNN with LeakyReLU activation for stable training.
* **Focus:** Robust Data Preprocessing and Feature Engineering.

---
## SECTION 1: ENVIRONMENT SETUP AND DATA PREPARATION

In [21]:
# --- Block 1: Library Imports and Setup ---

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam

# Set a random seed for reproducible results
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries imported successfully.")

Libraries imported successfully.


## SECTION 2: DATA PREPROCESSING (CLEANING, SCALING, ENCODING)

This step prepares the raw data for the neural network. It involves crucial steps like dropping corrupted rows, converting categorical text into numerical format (One-Hot Encoding), and normalizing numerical features (StandardScaler). This ensures model stability and high performance.

In [22]:
# --- Block 2: Data Preprocessing ---

# --- Configuration ---
FILE_PATH = 'parking_data_5000_records.csv'
epochs = 50

# --- 1. Data Loading (Robust Read to handle formatting issues) ---
try:
    # Use robust regex separator to handle mixed delimiters (comma, semicolon, space)
    df = pd.read_csv(FILE_PATH, sep=r'\s*[;,]\s*', engine='python', on_bad_lines='skip')

    # Strip whitespace from column names
    df.columns = df.columns.str.strip()

    # Handle potential bad headers by deleting the first row if it looks like a header
    if 'Date' not in df.columns and df.iloc[0].astype(str).str.contains('Date').any():
        df = df.iloc[1:].copy()
        df.columns = df.iloc[0]
        df = df[1:].reset_index(drop=True)
        df.columns.name = None

except Exception as e:
    print(f"ERROR: Could not read file. Check the file name and structure. Error: {e}")
    exit()

# Drop the date column, as its features are extracted into other columns
df = df.drop(['Date'], axis=1)


# --- 2. Data Preprocessing (Numeric Column Cleaning) ---

# Define feature groups
categorical_features = ['Day_of_Week', 'Weather', 'Lecture_Type']
numerical_features = ['Time_Hour', 'Temp_C', 'Enrollment_Ratio', 'Is_Weekend', 'Is_Exam_Week', 'Holiday_Effect']

# *CRITICAL FIX:* Convert numeric columns to float, coercing non-numeric to NaN
for col in numerical_features:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with any NaN values in critical numeric features
df = df.dropna(subset=numerical_features)
print(f"Data loaded successfully with {len(df)} clean records.")


# Separate features (X) and target (y)
X = df.drop(['Parking_Occupancy_Rate'], axis=1)
y = df['Parking_Occupancy_Rate']


# 2a. One-Hot Encoding for Categorical Features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_cats = encoder.fit_transform(X[categorical_features])
encoded_feature_names = encoder.get_feature_names_out(categorical_features)
encoded_df = pd.DataFrame(encoded_cats, columns=encoded_feature_names)

# 2b. Feature Scaling for Numerical Features (StandardScaler)
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X[numerical_features])
X_numeric_df = pd.DataFrame(X_numeric_scaled, columns=numerical_features)

# Combine all processed features
X_processed = pd.concat([X_numeric_df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

input_dim = X_train.shape[1]
print(f"Preprocessing complete. Total features for model: {input_dim}")

Data loaded successfully with 895 clean records.
Preprocessing complete. Total features for model: 23


## SECTION 3: DEEP LEARNING MODEL DEFINITION AND TRAINING

Here, we define the architecture of the Deep Neural Network (DNN). The model uses LeakyReLU activation for non-linearity and an Adam optimizer with Gradient Clipping to prevent the exploding gradients issue ('nan' values).

In [23]:
# --- Block 3: Model Definition and Training ---

# --- 3. Neural Network Model Definition ---
# This is a Deep Neural Network (DNN) Regression Model predicting a continuous rate (0.0 to 1.0).
model = Sequential([
    # Hidden Layer 1: Uses LeakyReLU for stable training (prevents 'nan' issues)
    Dense(128, input_shape=(input_dim,), name='Hidden_Layer_1'),
    LeakyReLU(alpha=0.01),

    # Dropout Layer: Prevents overfitting by randomly setting 20% of inputs to zero
    Dropout(0.2, name='Dropout_1'),

    # Hidden Layer 2:
    Dense(64, name='Hidden_Layer_2'),
    LeakyReLU(alpha=0.01),

    # Hidden Layer 3:
    Dense(32, name='Hidden_Layer_3'),
    LeakyReLU(alpha=0.01),

    # Output Layer: Sigmoid activation ensures the output is always between 0 and 1 (a percentage/rate)
    Dense(1, activation='sigmoid', name='Output_Layer')
])

# Define a custom optimizer with Gradient Clipping to prevent Exploding Gradients (CRITICAL FIX for 'nan')
custom_optimizer = Adam(learning_rate=0.001, clipvalue=1.0)

# Compile the model
# Loss: Mean Squared Error (MSE) is standard for regression
# Metrics: Mean Absolute Error (MAE) is easier to interpret as average percentage error
model.compile(optimizer=custom_optimizer,
              loss='mse',
              metrics=['mae'])

print("\n--- Model Architecture Summary ---")
model.summary()


# --- 4. Training the Model ---
print(f"\nTraining Model for {epochs} epochs...")
# The model learns to map input features (time, weather, exam status) to the parking occupancy rate.
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    validation_split=0.1, # Monitor performance on 10% of the training data
    verbose=1 # Show training progress
)
print("\nTraining complete.")


--- Model Architecture Summary ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Hidden_Layer_1 (Dense)          │ (None, 128)            │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_18 (LeakyReLU)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_2 (Dense)          │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_19 (LeakyReLU)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_3 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_20 (LeakyReLU)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,441 (52.50 KB)

 Trainable params: 13,441 (52.50 KB)

 Non-trainable params: 0 (0.00 B)


Training Model for 50 epochs...
Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - loss: 0.0877 - mae: 0.2545 - val_loss: 0.0218 - val_mae: 0.1202
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0153 - mae: 0.1011 - val_loss: 0.0027 - val_mae: 0.0430
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0037 - mae: 0.0482 - val_loss: 0.0013 - val_mae: 0.0299
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0024 - mae: 0.0393 - val_loss: 0.0011 - val_mae: 0.0279
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0021 - mae: 0.0359 - val_loss: 8.6068e-04 - val_mae: 0.0250
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0019 - mae: 0.0332 - val_loss: 7.6741e-04 - val_mae: 0.0227
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0018 - mae: 0.0340 - val_loss: 6.8325e-04 - val_mae: 0.0200
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0015 - mae: 0.0302 - val_loss: 5.5277e-04 - val_mae: 0.0179
Epoch 9/50
21/21 

## SECTION 4: MODEL EVALUATION AND PREDICTION

This final section evaluates the trained model on unseen test data to calculate performance metrics (MAE) and runs a scenario prediction to demonstrate the model's practical utility.

In [24]:
# --- Block 4: Evaluation and Prediction ---

# Retrieve MAE and Loss from the model evaluation
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\n--- 5. Model Evaluation on Test Data ---")
print(f"Test Loss (MSE): {loss:.4f}")
print(f"Test Mean Absolute Error (MAE): {mae:.4f}")

# Calculate human-readable accuracy and error percentages
accuracy_percent = (1 - mae) * 100
error_percent = mae * 100

print(f"\n--- Interpretation ---")
print(f"1. Model Accuracy: {accuracy_percent:.2f}% (Based on MAE)")
print(f"2. Average Error: The prediction is off by approximately {error_percent:.2f}% on average.")


# --- 6. Example Prediction for a High-Stress Scenario ---

# Scenario: Tuesday, 11:00 AM, Exam Week, High Enrollment (The ultimate stress test)
sample_input = pd.DataFrame([{
    'Time_Hour': 11,
    'Day_of_Week': 'Tuesday',
    'Is_Weekend': 0,
    'Weather': 'Sunny',
    'Temp_C': 25,
    'Lecture_Type': 'Morning_Lecture',
    'Enrollment_Ratio': 0.99,
    'Is_Exam_Week': 1,
    'Holiday_Effect': 0
}])

# Helper function to preprocess the sample data using the fitted scaler and encoder
def preprocess_sample(sample_df, scaler, encoder, numerical_features, categorical_features, column_order):
    # Scale numeric features
    sample_numeric_scaled = scaler.transform(sample_df[numerical_features])
    # Encode categorical features
    sample_encoded = encoder.transform(sample_df[categorical_features])

    # Combine and reindex to match the exact training column order (CRITICAL)
    sample_processed_array = np.concatenate([sample_numeric_scaled, sample_encoded], axis=1)

    # Create DataFrame, then reindex to ensure column order is identical to X_processed
    X_sample = pd.DataFrame(sample_processed_array, columns=encoder.get_feature_names_out(categorical_features).tolist() + numerical_features)
    X_sample = X_sample.reindex(columns=column_order, fill_value=0)
    return X_sample.head(1)

# Prepare the sample input
X_sample = preprocess_sample(
    sample_input,
    scaler,
    encoder,
    numerical_features,
    categorical_features,
    X_processed.columns
)

# Make the prediction
prediction = model.predict(X_sample, verbose=0)[0]
predicted_rate = prediction[0]

print(f"\n--- Scenario Prediction ---")
print(f"Input: Tuesday, 11:00 AM (Peak Time) during Exam Week")
print(f"Predicted Parking Occupancy Rate: {predicted_rate*100:.2f}%")


--- 5. Model Evaluation on Test Data ---
Test Loss (MSE): 0.0002
Test Mean Absolute Error (MAE): 0.0104

--- Interpretation ---
1. Model Accuracy: 98.96% (Based on MAE)
2. Average Error: The prediction is off by approximately 1.04% on average.

--- Scenario Prediction ---
Input: Tuesday, 11:00 AM (Peak Time) during Exam Week
Predicted Parking Occupancy Rate: 47.09%
